# 第三节 大语言模型LLMs，模版与聊天模型

## 大语言模型llm的常用参数与函数

In [ ]:
#输入三个模型各自的key

import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

os.environ["DASHSCOPE_API_KEY"] = "sk-9bcefecc91cb4a20b04c5cabce8963dc"

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""



### 参数 Temperature

Temperature: 用于调整随机从生成模型中抽样的程度，因此每次“生成”时，相同的提示可能会产生不同的输出。温度为 0 将始终产生相同的输出。温度越高随机性越大！主要用于控制创造力。

LLM中的一个耐人寻味的参数是 「温度」。「温度」是一个用于调整模型生成文本时创造性和多样性的超参数。「温度」是一个大于0的数值，通常在 0 到 1 之间。它影响模型生成文本时采样预测词汇的概率分布。当模型的「温度」较高时（如 0.8、1 或更高），模型会更倾向于从较多样且不同的词汇中选择，这使得生成的文本风险性更高、创意性更强，但也可能产生更多的错误和不连贯之处。而当「温度」较低时（如 0.2、0.3 等），模型主要会从具有较高概率的词汇中选择，从而产生更平稳、更连贯的文本。但此时，生成的文本可能会显得过于保守和重复。因此在实际应用中，需要根据具体需求来权衡选择合适的「温度」值。

https://zhuanlan.zhihu.com/p/666670367

1. 首先，让我们调用通义千问。

In [2]:
llm_tongyi_di=Tongyi(temperature=0.1)
llm_tongyi_zh=Tongyi(temperature=0.5)
llm_tongyi_ga=Tongyi(temperature=1)

In [3]:
prompt0="请写一首有关大理的四言诗"

In [4]:
print(llm_tongyi_di.invoke(prompt0))

大理风光，山川秀美。
洱海映月，苍山雪翠。
古道西风，瘦马寻梅。
玉龙蜿蜒，云影徘徊。


In [5]:
print(llm_tongyi_zh.invoke(prompt0))

大理风光，山川秀丽。
洱海波光，苍山雪霁。
古城悠悠，岁月静好。
风花雪月，心旷神怡。


In [6]:
print(llm_tongyi_ga.invoke(prompt0))

大理风光，山川秀丽。
洱海明珠，苍山玉带。
古道西风，瘦马行旅。
白墙青瓦，民风淳美。


### 参数streaming——流式输出

* 通义千问和星火似乎都不支持下面langchain中文文档中的流式输出模式，只有文心一言能用
https://python.langchain.com.cn/docs/modules/model_io/models/chat/streaming

* 由于发现zhipu ai有比较好的embeddings功能，可能使用zhipu ai作为向量数据库存储的主要模型，所以在此试试langchain的zhipu ai的流式输出模式。

In [12]:
from langchain_community.chat_models import ChatZhipuAI

llm_zhipu = ChatZhipuAI(
    temperature=0.5,
    api_key="a6ed0abe5933434b8ae29d44ed4997ba.frKjIq8FKLpKuIlq",
    model="glm-4",
)

In [18]:
stream = llm_zhipu.stream("帮我写一首有关西湖的歌")
full = next(stream)
for chunk in stream:
    full += chunk
full

AIMessageChunk(content='《梦绕西湖》\n\n（第一段）\n踏遍千山万水，独醉西湖美景\n碧波荡漾，风轻云淡，如诗如画如梦境\n烟雨江南，水墨丹青，浓墨重彩难画情\n\n（副歌）\n梦绕西湖，心恋江南水乡\n白蛇传奇，断桥残雪，千古传颂\n渔舟唱晚，夕阳映照，岁月静好\n此生此景，不负如来不负卿\n\n（第二段）\n漫步苏堤春晓，桃红柳绿映水间\n芳草萋萋，莺歌燕舞，春意盎然\n雷峰塔下，夕照金山，时光荏苒换新颜\n\n（副歌）\n梦绕西湖，心恋江南水乡\n白蛇传奇，断桥残雪，千古传颂\n渔舟唱晚，夕阳映照，岁月静好\n此生此景，不负如来不负卿\n\n（桥段）\n碧水映晴空，风起云涌，似仙子舞动长袖\n诗酒趁年华，共度良辰，让时光倒流\n江南烟雨，梦回西湖，那年的约定\n手牵手，共赴一场美丽的邂逅\n\n（副歌）\n梦绕西湖，心恋江南水乡\n白蛇传奇，断桥残雪，千古传颂\n渔舟唱晚，夕阳映照，岁月静好\n此生此景，不负如来不负卿\n\n（尾声）\n梦绕西湖，恋恋不舍，离别总在日出时\n愿岁月静好，佳人依旧，共度余生\n让这美景，化作永恒，留在心间\n梦中相见，那片美丽的西湖山水', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'token_usage': {'prompt_tokens': 11, 'completion_tokens': 367, 'total_tokens': 378}, 'model_name': 'glm-4'}, id='run--c210eb0a-8787-4eff-ad8c-25973c70dfc9')

In [9]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm_wx_liu = QianfanLLMEndpoint(streaming=True, callbacks=[StreamingStdOutCallbackHandler()],temperature=0.1)

In [11]:
resp = llm_wx_liu.invoke("帮我写一首有关西湖的歌")

[INFO] [03-04 20:22:55] openapi_requestor.py:316 [t:3588]: requesting llm api endpoint: /chat/eb-instant


当然可以，以下是一首关于西湖的歌：

西湖之歌

(诗词)

水光潋滟晴方好，山色空蒙雨亦奇。
欲把西湖比西子，淡妆浓抹总相宜。

(副歌)

西湖之歌，婉约又豪放，
碧波荡漾，情深意长。
柳丝轻舞，花儿笑开，
西湖之美，让人心醉。

(歌词)

第一节：
晨曦微露，西湖醒来，
薄雾轻绕，如梦如幻。
白鹭低飞，鱼儿跃起，
碧波荡漾，心旷神怡。

副歌：
西湖之歌，婉约又豪放，
碧波荡漾，情深意长。
月下西湖，静谧又神秘，
夜色西湖，如诗如画。

第二节：
雷峰塔倒，白娘子的传说，
断桥残雪，千古传颂。
苏堤春晓，花开如画，
三潭印月，美不胜收。

副歌：
西湖之歌，婉约又豪放，
碧波荡漾，情深意长。
风吹湖面，涟漪荡漾，
心随西湖，飘向远方。

第三节和尾声：
西湖之美，四季如画，
春花秋月，夏阳冬雪。
走过岁月，西湖依旧，
我心中的西湖之歌，永远唱响。

这首歌描绘了西湖的美丽景色和传说，表达了对西湖的深深喜爱和向往。希望你喜欢！

### 函数get_num_tokens————计算token数量

In [21]:
# 需要安装PyTorch
llm_tongyi_ga.get_num_tokens("fasfdsafreqwrwe")

/opt/anaconda3/envs/langchain_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


KeyboardInterrupt: 

### 函数generate————多个提示词的支持

* 注意对应信息获取方式
* 该方法依然能获取到tokens

In [23]:
res0=llm_tongyi_di.generate(["写一首四句的诗","写一幅对联"])

In [24]:
res0

LLMResult(generations=[[Generation(text='春来花开满园香，\n夏至蝉鸣绿荫长。\n秋风送爽叶舞黄，\n冬雪皑皑炉火旺。', generation_info={'finish_reason': 'stop', 'request_id': '8f94717f-e9f5-98c5-824a-6d904db702f0', 'token_usage': {'input_tokens': 14, 'output_tokens': 31, 'total_tokens': 45, 'prompt_tokens_details': {'cached_tokens': 0}}})], [Generation(text='上联：春风得意马蹄疾  \n下联：岁月峥嵘人未老  \n\n横批：前程似锦', generation_info={'finish_reason': 'stop', 'request_id': 'dca26296-bc04-9275-a843-ae8451219968', 'token_usage': {'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38, 'prompt_tokens_details': {'cached_tokens': 0}}})]], llm_output={'model_name': 'qwen-plus'}, run=[RunInfo(run_id=UUID('a843ad3b-f5af-4a2e-908a-f03afe606637')), RunInfo(run_id=UUID('c30477a2-3eb4-46ac-89a9-22a89c397cea'))], type='LLMResult')

In [25]:
res0.generations[0][0].generation_info

{'finish_reason': 'stop',
 'request_id': '8f94717f-e9f5-98c5-824a-6d904db702f0',
 'token_usage': {'input_tokens': 14,
  'output_tokens': 31,
  'total_tokens': 45,
  'prompt_tokens_details': {'cached_tokens': 0}}}

### classwork-1

1. 让大模型帮忙写一幅对联，用get_num_tokens获得此提示词的token数量

2. 改变模型的温度，看看对联是否有不一样？

3. 通过流式输出上述对联的结果

4. 通过列表定义任意几个提示词，通过generate获得多个提示词的生成结果, 并获得这些结果的total_tokens

In [28]:
# 模型低、中、高的 temperature 生成的结果
low_temp_result = llm_tongyi_di.invoke("帮我写一幅对联")
mid_temp_result = llm_tongyi_zh.invoke("帮我写一幅对联")
high_temp_result = llm_tongyi_ga.invoke("帮我写一幅对联")

# 格式化输出
print("=== 低温度 (Low Temperature) ===")
print(low_temp_result)
print("\n=== 中温度 (Medium Temperature) ===")
print(mid_temp_result)
print("\n=== 高温度 (High Temperature) ===")
print(high_temp_result)

=== 低温度 (Low Temperature) ===
上联：春风得意马蹄疾  
下联：岁月峥嵘人未老  

横批：前程似锦

=== 中温度 (Medium Temperature) ===
上联：春风得意马蹄疾  
下联：岁月峥嵘人未老  

横批：前程似锦

=== 高温度 (High Temperature) ===
上联：春风得意马蹄疾  
下联：岁月峥嵘人未老  

横批：前程似锦


In [33]:
classwork_res = llm_tongyi_di.generate(["帮我写一幅对联","写一首关于成都的诗"])
classwork_res

LLMResult(generations=[[Generation(text='上联：春风得意马蹄疾  \n下联：岁月峥嵘人未老  \n\n横批：前程似锦', generation_info={'finish_reason': 'stop', 'request_id': 'f83616c5-8bcc-90a8-a18f-ae1a078b999a', 'token_usage': {'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39, 'prompt_tokens_details': {'cached_tokens': 0}}})], [Generation(text='锦官城外柏森森，遥望西岭雪皑皑。\n成都平原风光好，花重锦官城自开。\n\n草堂遗址今犹在，杜甫诗魂永不衰。\n武侯祠前思贤相，青羊宫里祈年来。\n\n春熙路上人潮涌，宽窄巷中古韵来。\n火锅飘香满街巷，川剧变脸惊四海。\n\n夜幕降临锦江畔，灯火辉煌映水来。\n九天开出一成都，万户千门入画图。', generation_info={'finish_reason': 'stop', 'request_id': '5117a991-bae9-9dd6-adcf-78934d2022af', 'token_usage': {'input_tokens': 14, 'output_tokens': 114, 'total_tokens': 128, 'prompt_tokens_details': {'cached_tokens': 0}}})]], llm_output={'model_name': 'qwen-plus'}, run=[RunInfo(run_id=UUID('9c490e80-2707-4e1d-8983-5263c4db34a0')), RunInfo(run_id=UUID('47da0b9f-676f-42f8-a5fe-bb1655dc6ce5'))], type='LLMResult')

In [49]:
# 查看tokens
type(classwork_res.generations[0][0])
classwork_res.generations[0][0].generation_info['token_usage']

{'input_tokens': 13,
 'output_tokens': 26,
 'total_tokens': 39,
 'prompt_tokens_details': {'cached_tokens': 0}}

## 模版的使用

### 简单模版

语言模型以文本作为输入，这段文本通常被称为提示（Prompt）。通常情况下，这不仅仅是一个硬编码的字符串，而是模板、示例和用户输入的组合。LangChain提供了多个类和函数，以便轻松构建和处理提示。

提示模板是指一种可复制的生成提示的方式。它包含一个文本字符串（模板），可以从最终用户处接收一组参数并生成提示。提示模板可能包含以下内容：

* 对语言模型的指令
* 少量示例，以帮助语言模型生成更好的回复
* 对语言模型的问题

原文链接： https://machinelearning.blog.csdn.net/article/details/131988052

In [50]:
from langchain import PromptTemplate

template = """
我希望你能充当新公司的命名顾问。
一个生产{product}的公司的好名字是什么？
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt.format(product="彩色袜子")


'\n我希望你能充当新公司的命名顾问。\n一个生产彩色袜子的公司的好名字是什么？\n'

In [52]:
print(llm_tongyi_zh.invoke(prompt.format(product="彩色袜子")))

为一家生产彩色袜子的公司命名时，我们希望名字能够体现色彩、趣味和时尚感，同时易于记忆和传播。以下是一些创意名称供你参考：

### 1. **ColorSocks**
   - 简洁明了，直接传达“彩色袜子”的核心理念。
   - 可以搭配标语如“让每一天都充满色彩”。

### 2. **RainbowFeet**
   - “Rainbow”象征丰富的颜色，“Feet”则点明产品用途。
   - 给人活泼、年轻的感觉。

### 3. **Socktastic**
   - 结合“sock”（袜子）与“fantastic”（奇妙的），创造一个有趣且独特的词汇。
   - 强调产品的独特性和吸引力。

### 4. **Chrom Sox**
   - “Chrom”来源于“chromatic”（色彩缤纷的），与“sox”（袜子的简写）结合。
   - 科技感与艺术感并存。

### 5. **HappyHeels**
   - 着重于穿着者的心情，“Happy”传递积极情绪，“Heels”可以理解为脚后跟或整体脚部。
   - 温暖而亲切。

### 6. **Artful Socks**
   - 将袜子比作艺术品，展现设计的独特性。
   - 非常适合强调个性化和创意图案的品牌。

### 7. **Pigment Feet**
   - “Pigment”意为颜料，突出了袜子的颜色丰富性。
   - 名字现代且有深度。

### 8. **BriteSocks**
   - “Brite”是“bright”（明亮的）的变体拼写，寓意鲜艳的颜色。
   - 易记且富有活力。

### 9. **Sole Colors**
   - “Sole”既指“灵魂”，也指“鞋底”，双关语增加了趣味性。
   - 表达每双袜子都有其独特的灵魂和色彩。

### 10. **Crazy Threads**
   - “Threads”泛指衣物纤维，“Crazy”体现大胆的设计风格。
   - 适合走潮流路线的品牌。

如果你需要更具体的建议，比如目标市场、品牌定位或其他元素，请告诉我！这样我可以进一步优化名字方案。


In [57]:
# 多个输入变量的示例提示
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="给我讲一个{adjective}的关于{content}的中文笑话。"
)
multiple_input_prompt.format(adjective="有趣的", content="小鸡")

'给我讲一个有趣的的关于小鸡的中文笑话。'

In [58]:
print(llm_tongyi_zh.invoke(multiple_input_prompt.format(adjective="有趣的", content="小鸡")))


当然可以！这是一个关于小鸡的有趣笑话：

有一天，一只小鸡去参加考试。考卷上第一题是：“蛋是从哪里来的？”

小鸡想了想，在答案栏里写道：“从我肚子里出来的。”

老师改卷时看到这个答案，哭笑不得，便在旁边批注：“错！请回去问你的妈妈。”

小鸡回家后把老师的评语给鸡妈妈看，鸡妈妈看完后严肃地对小鸡说：“孩子，以后这种问题你就直接写‘树上掉下来的’，明白了吗？”

希望你喜欢这个笑话！😄


###  包含例子的模版

向模板添加Few Shot示例
Few Shot示例是一组可以帮助语言模型生成更好响应的示例。要使用Few Shot示例生成提示，可以使用FewShotPromptTemplate。此类接受PromptTemplate和Few Shot示例列表。然后，它使用Few Shot示例格式化提示模板。

在下面示例中，我们将创建一个生成单词反义词的提示：

In [62]:
from langchain import PromptTemplate, FewShotPromptTemplate

# 首先，创建Few Shot示例列表
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "正常", "antonym": "反常"},
]

# 接下来，我们指定用于格式化示例的模板。
# 我们使用`PromptTemplate`类来实现这个目的。

example_formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

# 最后，创建`FewShotPromptTemplate`对象。
few_shot_prompt = FewShotPromptTemplate(
    # 这些是我们要插入到提示中的示例。
    examples=examples,
    # 这是我们在将示例插入到提示中时要使用的格式。
    example_prompt=example_prompt,
    # 前缀是出现在提示中示例之前的一些文本。
    # 通常，这包括一些说明。
    prefix="Give the antonym of every input\n",
    # 后缀是出现在提示中示例之后的一些文本。
    # 通常，这是用户输入的地方。
    suffix="Word: {input}\nAntonym: ",
    # 输入变量是整个提示期望的变量。
    input_variables=["input"],
    # 示例分隔符是我们将前缀、示例和后缀连接在一起的字符串。
    example_separator="\n",
)

# 现在，我们可以使用`format`方法生成一个提示。
print(few_shot_prompt.format(input="big"))



Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: 正常
Antonym: 反常

Word: big
Antonym: 


In [63]:
llm_tongyi_zh.invoke(few_shot_prompt.format(input="big"))

'small'

In [ ]:
#    {"word": "正常", "antonym": "反常"} 在加入该中文例子后，会有中文对应反义词输出
llm_tongyi_di(few_shot_prompt.format(input="美丽"))

'丑陋'

### classwork-2

1. 通过引入带 __变量__ 的 __模板__ ，控制对联的类型，以此输出满足不同需求的对联

2. 使用带 __例子__ 的 __模板__ 实现一个自动对对联工具，要求你给出上联，大模型给出对应下联，例子自己百度
   

In [65]:
# 1.带变量的模版
from langchain import PromptTemplate
template = """
你是一个中文文人，我希望你能帮我写一幅关于{input}的对联。
"""

prompt = PromptTemplate(
    input_variables=["input"],
    template=template,
)

prompt.format(input="成都")

'\n你是一个中文文人，我希望你能帮我写一幅关于成都的对联。\n'

In [67]:
# 使用大模型 回答 提示词模版的提示内容
print(llm_tongyi_zh.invoke(prompt.format(input="重庆")))

上联：山城雾绕，巴渝古韵千秋画  
下联：江水流长，重庆新风万里歌  

横批：山河壮丽  

这幅对联中，"山城雾绕"描绘了重庆多雾的自然特色，"巴渝古韵千秋画"体现了重庆悠久的历史文化；"江水流长"指长江与嘉陵江在重庆交汇，象征城市的发展源远流长，"重庆新风万里歌"则展现了现代重庆蓬勃发展的新风貌。横批"山河壮丽"概括赞美了重庆美丽的山水景色。


In [ ]:
# 2.使用带 例子 的 模版 实现一个自动对对联工具，要求你给出上联，大模型给出对应下联
from langchain import PromptTemplate, FewShotPromptTemplate
# 首先，创建Few Shot示例列表
examples = [
    {"上联":"古典精华凝诗句","下联":"现代风貌入画屏", "横批":"文化传承"},
    {"上联":"月明点滴窗前影","下联":"风清摇曳树下声", "横批":"自然之美"},
    {"上联":"春风化雨润心田","下联":"夏日炎炎照大地", "横批":"四季交替"},
]

# 接下来，我们指定用于格式化示例的模板。
# 我们使用`PromptTemplate`类来实现这个目的。
example_formatter_template = """上联: {上联}
下联: {下联}
横批: {横批}
"""
example_prompt = PromptTemplate(
    input_variables=["上联", "下联", "横批"],
    template=example_formatter_template,
)
# 最后，创建`FewShotPromptTemplate`对象。
few_shot_prompt = FewShotPromptTemplate(
    # 这些是我们要插入到提示中的示例。
    examples=examples,
    # 这是我们在将示例插入到提示中时要使用的格式。
    example_prompt=example_prompt,
    # 前缀是出现在提示中示例之前的一些文本。
    # 通常，这包括一些说明。
    prefix="你是一位中国的文人，给我写一幅对联\n",
    # 后缀是出现在提示中示例之后的一些文本。
    # 通常，这是用户输入的地方。
    suffix="上联: {input}\n下联: 横批: ",
    # 输入变量是整个提示期望的变量。
    input_variables=["input"],
    # 示例分隔符是我们将前缀、示例和后缀连接在一起的字符串。
    example_separator="\n",
)
# 现在，我们可以使用`format`方法生成一个提示。
print(few_shot_prompt.format(input="古典精华凝诗句"))

你是一位中国的文人，给我写一幅对联

上联: 古典精华凝诗句
下联: 现代风貌入画屏
横批: 文化传承

上联: 月明点滴窗前影
下联: 风清摇曳树下声
横批: 自然之美

上联: 春风化雨润心田
下联: 夏日炎炎照大地
横批: 四季交替

上联: 古典精华凝诗句
下联: 


In [70]:
print(llm_tongyi_zh.invoke(few_shot_prompt.format(input="烟锁池塘柳")))

雾罩树林松

横批: 景色朦胧 

解析：上联“烟锁池塘柳”描绘出烟雾缭绕着池塘边的柳树，营造出一种朦胧的意境。下联“雾罩树林松”则对应地写出雾气笼罩着树林中的松树，与上联在景象上对仗工整，都展现了一种模糊而又静谧的大自然画面。“景色朦胧”作为横批，恰如其分地概括了这种因烟雾而产生的独特美感。


## 聊天模型-chat_models

聊天模型是语言模型的一种变体。虽然聊天模型在内部使用语言模型，但它们公开的接口略有不同。它们不是提供一个“输入文本，输出文本”的API，而是提供一个以“聊天消息”作为输入和输出的接口。 聊天模型的API还比较新，因此我们仍在确定正确的抽象层次。本问将介绍如何开始使用聊天模型，该接口是基于消息而不是原始文本构建的

通过向聊天模型传递一个或多个消息，可以获取聊天完成的结果。响应将是一个消息。LangChain目前支持的消息类型有AIMessage、HumanMessage、SystemMessage和ChatMessage，其中ChatMessage接受一个任意的角色参数。大多数情况下，我们只需要处理HumanMessage、AIMessage和SystemMessage。
                        
原文链接：https://blog.csdn.net/hy592070616/article/details/131924904

### 通义千问，星火，文心一言的聊天模型实现

#### 通义千问

当前langchain版本（0.1.9）通义千问的流式接口有问题，暂时建议不用流式

https://python.langchain.com/docs/integrations/chat/tongyi

In [71]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

In [72]:
from langchain_core.messages import HumanMessage

chat_ty = ChatTongyi(
    streaming=True,
)
res = chat_ty.stream([HumanMessage(content="hi")], streaming=True)

In [73]:
res = chat_ty.stream([HumanMessage(content="帮我写副对联")], streaming=True)

In [74]:
try:
    for chunk in res:
        print(chunk)
except TypeError as e:
    print("")

content='上' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='联' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='：' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='春风送暖入' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='屠苏\n下' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='联：瑞雪' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='迎春盈梅' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='香' additional_kwargs={} response_metadata={} id='run--c582963d-b03b-47bc-8054-496e63be543c'
content='' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'request_id': 'f733cbca-dcf4-9d14-bc48-d2397784342a', 'token_usage': {'input_tokens': 13, 'output_toke

In [75]:
chat_ty = ChatTongyi()
chat_ty.invoke([HumanMessage(content="帮我写副对联")])

AIMessage(content='上联：春风送暖入屠苏\n下联：瑞雪迎春兆丰年', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '3cbc5654-bef5-9e84-9b96-6ba2931a7788', 'token_usage': {'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--db843d66-f6c8-40a8-b337-d2c9747ec62c-0')

#### 星火大模型

https://python.langchain.com/docs/integrations/chat/sparkllm

In [6]:
chat_xh = ChatSparkLLM()
message = HumanMessage(content="Hello")
chat_xh.invoke([message])

AIMessage(content='Hello! How can I assist you today?')

In [7]:
# 流式调用
chat_xh_s = ChatSparkLLM(streaming=True)
for chunk in chat_xh_s.stream("Hello!"):
    print(chunk.content, end="")

Hello! How can I help you today?

### 百度文心一言

https://python.langchain.com/docs/integrations/chat/baidu_qianfan_endpoint

In [8]:
chat_wx = QianfanChatEndpoint(streaming=True)
messages = [HumanMessage(content="Hello")]
chat_wx.invoke(messages)


[INFO] [03-04 22:27:40] openapi_requestor.py:316 [t:8028]: requesting llm api endpoint: /chat/eb-instant
[INFO] [03-04 22:27:40] oauth.py:207 [t:8028]: trying to refresh access_token for ak `og6mWr***`
[INFO] [03-04 22:27:40] oauth.py:220 [t:8028]: sucessfully refresh access_token


AIMessage(content='您好！有什么我可以帮助您的吗？')

In [9]:
try:
    for chunk in chat_wx.stream(messages):
        print(chunk.content, end="", flush=True)
except TypeError as e:
    print("")

[INFO] [03-04 22:27:41] openapi_requestor.py:316 [t:8028]: requesting llm api endpoint: /chat/eb-instant


您好！有什么我可以帮助您的吗？


### 聊天模型的使用

In [76]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]  

In [ ]:
chat_wx.invoke(messages)

In [77]:
chat_ty.invoke(messages)

AIMessage(content="J'adore programmer.", additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'c7b86e87-5adc-9687-a72e-41135d32958d', 'token_usage': {'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--1764d6bd-2679-4371-8ccc-b1b2191ab4ad-0')

In [78]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat_ty.generate(batch_messages)
result


LLMResult(generations=[[ChatGeneration(text="J'adore programmer.", generation_info={'finish_reason': 'stop', 'request_id': '73cc39b0-121e-9bdd-89bf-81cc990d4f0a', 'token_usage': {'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'prompt_tokens_details': {'cached_tokens': 0}}}, message=AIMessage(content="J'adore programmer.", additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '73cc39b0-121e-9bdd-89bf-81cc990d4f0a', 'token_usage': {'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--9265307d-3395-4f30-8d04-f7dea1412ed3-0'))], [ChatGeneration(text="J'adore l'intelligence artificielle.", generation_info={'finish_reason': 'stop', 'request_id': '876152e4-9dc7-9a91-af84-bb06bc8ab5cd', 'token_usage': {'input_tokens': 29, 'output_tokens': 12, 'total_tokens': 41, 'prompt_tokens_details': {'cached_tokens': 0}}}, message=AIMessage(content="J'adore l'intelligence arti

In [79]:
result.llm_output

{'model_name': 'qwen-turbo'}

### 模版在聊天模型中的使用

我们可以使用模板来构建MessagePromptTemplate。我们可以从一个或多个MessagePromptTemplate构建一个ChatPromptTemplate。我们还可以使用ChatPromptTemplate的format_prompt方法，它将返回一个PromptValue，我们可以将其转换为字符串或消息对象，具体取决于我们是否希望将格式化后的值作为输入传递给LLM或Chat模型的输入。为了方便起见，模板上公开了一个from_template方法。如果您要使用此模板，代码如下所示：

In [80]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 获取格式化后的消息的聊天完成结果
chat_ty.invoke(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())


AIMessage(content="J'adore programmer.", additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '58b630d2-cb12-9bc5-bb07-ebabcbbf105c', 'token_usage': {'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--d6f1a732-ac66-4ae8-8d66-f987342751b2-0')

### classwork-3

1. 基于聊天模型实现对对联工具，人给出上联，模型给出下联

2. 使用模板实现用不同语言的对对联工具